In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [15]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset,DataLoader
import torch.utils.data as data_utils
from sklearn import metrics

In [16]:
from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard
!rm -rf ./logs/ 
import shutil
import os
import time

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [17]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reading CSV files

In [18]:
try: 
    os.mkdir('/content/Best_model/')
except OSError as error: 
    print(error)

[Errno 17] File exists: '/content/Best_model/'


In [32]:
Facebook = pd.read_csv('/content/Facebook.csv',index_col=[0])
Instagram = pd.read_csv('/content/Instagram.csv',index_col=[0])
Twitter = pd.read_csv('/content/Twitter.csv',index_col=[0])
# All = pd.concat([Facebook,Instagram,Twitter],ignore_index=True)
# del All['QoE']

FB_QoE = Facebook['QoE']
IN_QoE = Instagram['QoE']
TW_QoE = Twitter['QoE']
QoEs = {'Facebook':FB_QoE,'Twitter':TW_QoE,'Instagram':IN_QoE}

del Facebook['QoE']
del Twitter['QoE']
del Instagram['QoE']

In [33]:
FB_train = torch.Tensor(np.array(Facebook))
IN_train = torch.Tensor(np.array(Instagram))
TW_train = torch.Tensor(np.array(Twitter))


In [31]:
print(FB_train.shape)
print(IN_train.shape)
print(TW_train.shape)


torch.Size([2562, 23])
torch.Size([2539, 23])
torch.Size([2706, 23])


In [9]:
# Fb_Train, Fb_val = torch.utils.data.random_split(FB_train,[int(len(FB_train)*0.9),len(FB_train)-(int(len(FB_train)*0.9))])
# In_Train, In_val = torch.utils.data.random_split(IN_train,[int(len(IN_train)*0.9),len(IN_train)-(int(len(IN_train)*0.9))])
# Tw_Train, Tw_val = torch.utils.data.random_split(TW_train,[int(len(TW_train)*0.9),len(TW_train)-(int(len(TW_train)*0.9))])


Fb_Train = FB_train[:int(len(FB_train)*0.9)]
Fb_val = FB_train[int(len(FB_train)*0.9):]

In_Train = IN_train[:int(len(IN_train)*0.9)]
In_val = IN_train[int(len(IN_train)*0.9):]

Tw_Train = FB_train[:int(len(TW_train)*0.9)]
Tw_val = FB_train[int(len(TW_train)*0.9):]

In [13]:
len(Fb_Train)
# len(Fb_val)

2305

In [20]:
Fb_val_loader = data_utils.DataLoader(Fb_val, batch_size = 100, shuffle = False)
Tw_val_loader = data_utils.DataLoader(Tw_val, batch_size = 100, shuffle = False)
In_val_loader = data_utils.DataLoader(In_val, batch_size = 100, shuffle = False)

In [21]:
class Net(nn.Module):
    def __init__(self,l1=16,l2=10):
        super(Net,self).__init__() 
        self.encoder = nn.Sequential(
            nn.Linear(23, l1), # (N, 784) -> (N, 128)
            nn.ReLU(),
            nn.Linear(l1, l2)
            )
        
        self.decoder = nn.Sequential(
            nn.Linear(l2, l1),
            nn.ReLU(),
            nn.Linear(l1, 23),
            nn.ReLU()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    def encode(self,x):
      return self.encoder(x)  

In [22]:
config = {
        "Weight_decay": [0.1,0.01,0.001],
        "lr": [0.01,0.1],
        # "batch_size": [100,128,200]
        "batch_size": [16,32,64,128]
    }

In [23]:
def evaluate(model,val_dataloader):

    model.eval()
    criterion = nn.MSELoss()
    losses = 0

    with torch.no_grad():
      for img in val_dataloader:

        recon = model(img)
        loss = criterion(recon, img)
        losses += loss.item()        

    return losses / len(val_dataloader)

In [24]:
def Tune(config,data,val_loader,Platform):
  start = time.time()
  print(f"-------  Platform is : {Platform}-------")
  with open(f'/content/Best_model/{Platform}_summary.txt','w') as f:
    f.write(f'Start Local Time: {time.ctime()}\n')
  num_epochs = 20
  Principle_Val = float('inf')
  criterion = nn.MSELoss()
  # model = Net()
  for lr in config['lr']:
    for w_d in config['Weight_decay']:
      for b_s in config['batch_size']:
        model = Net()
        optimizer = torch.optim.Adam(model.parameters(),
                          lr=lr, 
                        weight_decay= w_d)
        Train = data_utils.DataLoader(data, batch_size = b_s, shuffle = False)
        Best_loss = float('inf')
        train_start = time.time()
        for epoch in range(num_epochs):
          losses = 0.0
          for img in Train:

            optimizer.zero_grad()
            recon = model(img)
            loss = criterion(recon, img)

            loss.backward()
            optimizer.step()
            losses += loss.item()
         
          Val_loss = evaluate(model,val_loader)
          if Best_loss > Val_loss:
            Best_loss = Val_loss
            Best_epoch = epoch
            # torch.save(model.state_dict(), f'/content/models/lr{lr}_wd{w_d}_bs{b_s}.pth')
          model.train()

          # print("lr : ", lr,"  batch_size : " ,b_s,"  Weight_decay : ",w_d)
          # print(f'Epoch:{epoch+1}, Training Loss:{losses/(len(Train)*b_s):.4f}, Validtion Loss: {Val_loss} ')
        if Principle_Val > Best_loss:
            Principle_Val = Best_loss
            Principle_lr = lr
            Principle_wd = w_d
            Principle_bs = b_s
            Principle_epoch = Best_epoch
            torch.save(model.state_dict(), f'/content/Best_model/{Platform}_Best.pth')
        with open(f'/content/Best_model/{Platform}summary.txt','a') as f:
            f.write(f'lr : {lr}\tw_d : {w_d}\tb_s : {b_s}\tepoch : {Best_epoch}\tBest_Loss : {Best_loss}\n')
            print(f'lr : {lr}\tw_d : {w_d}\tb_s : {b_s}\tepoch : {Best_epoch}\tBest_Loss : {Best_loss}\tTime Taken : {time.time()-train_start}')
  
  with open(f'/content/Best_model/{Platform}_summary.txt','a') as f:
    f.write(f'Principle_lr : {Principle_lr}\tPrinciple_w_d : {Principle_wd}\tb_s : {Principle_bs}\tPrinciple_epoch : {Principle_epoch}\tPrinciple_Loss : {Principle_Val}\n')
    f.write(f'End Local Time: {time.ctime()}\n')
    Total_Time = time.time() - start
    f.write(f'Total Time : {Total_Time} s')
    print(f'Principle_lr : {Principle_lr}\tPrinciple_w_d : {Principle_wd}\tb_s : {Principle_bs}\tPrinciple_epoch : {Principle_epoch}\tPrinciple_Loss : {Principle_Val}')
    print("Total Time : ",Total_Time," s")
    print('\n\n')


In [25]:
Tune(config,Fb_Train,Fb_val_loader,"Facebook")
Tune(config,Tw_Train,Tw_val_loader,"Twitter")
Tune(config,In_Train,In_val_loader,"Instagram")

-------  Platform is : Facebook-------
lr : 0.01	w_d : 0.1	b_s : 16	epoch : 19	Best_Loss : 18.27100435892741	Time Taken : 2.9256129264831543
lr : 0.01	w_d : 0.1	b_s : 32	epoch : 19	Best_Loss : 19.90953191121419	Time Taken : 1.3924791812896729
lr : 0.01	w_d : 0.1	b_s : 64	epoch : 19	Best_Loss : 26.83017412821452	Time Taken : 0.8009390830993652
lr : 0.01	w_d : 0.1	b_s : 128	epoch : 16	Best_Loss : 44.890228271484375	Time Taken : 0.4722912311553955
lr : 0.01	w_d : 0.01	b_s : 16	epoch : 18	Best_Loss : 49.925576527913414	Time Taken : 2.6016907691955566
lr : 0.01	w_d : 0.01	b_s : 32	epoch : 19	Best_Loss : 139.7996368408203	Time Taken : 1.3926677703857422
lr : 0.01	w_d : 0.01	b_s : 64	epoch : 19	Best_Loss : 23.481685002644856	Time Taken : 0.7724993228912354
lr : 0.01	w_d : 0.01	b_s : 128	epoch : 19	Best_Loss : 41.710774739583336	Time Taken : 0.4715428352355957
lr : 0.01	w_d : 0.001	b_s : 16	epoch : 18	Best_Loss : 29.308215459187824	Time Taken : 2.597801923751831
lr : 0.01	w_d : 0.001	b_s : 32	

In [37]:
def Extract_features(model,Data,model_path):
  model.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
  model.eval()
  Extracted = model.encode(Data)
  print(Extracted.shape)
  return Extracted

Model = Net()
Platforms = {'Facebook':FB_train,'Twitter':TW_train,'Instagram':IN_train}
for platform in Platforms.keys():
  path = f'/content/Best_model/{platform}_Best.pth'
  Extracted_features = Extract_features(Model,Platforms[platform],path)
  DF = pd.DataFrame(Extracted_features.detach().numpy())
  print("Samples match : ",len(QoEs[platform]) == len(DF),"\n")
  DF["QoE"] = QoEs[platform]
  DF.to_csv(f'/content/{platform}_Bottleneck.csv')

torch.Size([2562, 10])
Samples match :  True 

torch.Size([2706, 10])
Samples match :  True 

torch.Size([2539, 10])
Samples match :  True 

